# 拡散モデル

## モデル構築

In [31]:
# 拡散モデル
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn

# 学習データ
x = torch.tensor([[i] for i in range(1,11)], dtype=torch.float32)

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + 1, hidden_dim),  # t埋め込みを考慮して入力次元を +1
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)  # 出力は1次元
        )
    
    def forward(self, x, t):
        # tをログスケールで簡易的に埋め込む (1Dデータに適用)
        t_embed = torch.log1p(t.float()).unsqueeze(1)  # tを(バッチサイズ, 1)に変換
        x_t = torch.cat([x, t_embed], dim=1)  # xとt_embedを結合
        return self.model(x_t)

# モデル作成
input_dim = 1  # 入力次元 (例: xは3次元データ)
hidden_dim = 32  # 隠れ層のユニット数
model = MLP(input_dim=input_dim, hidden_dim=hidden_dim)

# テスト用データ
# x = torch.randn(10, input_dim)  # 10個の3次元の入力
t = torch.randint(1, 100, (10,))  # 10個の時間ステップ (整数)
y = model(x, t)  # 順伝播
print(y.shape)  # (10, 1)


torch.Size([10, 1])


In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# =====================================
# データ準備
# =====================================
# 非常に小さなデータセット x_0: [[1],[2],[3],[4],[5],[6],[7],[8],[9],[10]]
x0 = torch.tensor([[i] for i in range(1,11)], dtype=torch.float32)  # shape: (10,1)

# デバイス設定
device = 'cuda' if torch.cuda.is_available() else 'cpu'
x0 = x0.to(device)

# =====================================
# 拡散パラメータ設定
# =====================================
T = 1000  # ステップ数
beta_start = 0.0001
beta_end = 0.02
betas = torch.linspace(beta_start, beta_end, T, device=device) # (T,)
alphas = 1 - betas
alpha_bars = torch.cumprod(alphas, dim=0)  # (T,)

# =====================================
# q(x_t|x_0) 関数の定義
# =====================================
def q_sample(x_0, t):
    """
    x_0 からステップ t (1 <= t <= T) まで拡散したサンプル x_t を返す。
    同時に使用したノイズ eps も返す。
    """
    t_idx = t - 1
    alpha_bar_t = alpha_bars[t_idx]  # スカラー
    # eps ~ N(0, I)
    eps = torch.randn_like(x_0)
    x_t = torch.sqrt(alpha_bar_t) * x_0 + torch.sqrt(1 - alpha_bar_t) * eps
    return x_t, eps

# =====================================
# モデル定義 (3～4層程度のMLP)
# 入力: x_t (1次元), t (スカラー) -> 結合 -> (x_t, t_embed)
# 出力: eps_pred (1次元)
# =====================================
class DiffusionMLP(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=32):
        super().__init__()
        # input_dim=2は x_t と t_embed (あとでtを処理) を入れるため
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, x_t, t):
        # 簡易的な t の埋め込み: log(1+t) を使用
        t_embed = torch.log1p(t.float().unsqueeze(1))  # (batch,1)
        inp = torch.cat([x_t, t_embed], dim=1)         # (batch,2)
        eps_pred = self.fc(inp)
        return eps_pred

model = DiffusionMLP().to(device)

# =====================================
# 学習設定
# =====================================
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

num_epochs = 2000  # 短いデータなので多めに回す
losses = []

# =====================================
# 学習ループ
# =====================================
# コメント付きで説明
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # 1. ランダムに t をサンプリング
    #   全データ x0 に対して同じ t を適用しても良いが、ここは一斉に学習
    batch_size = x0.size(0)  
    t = torch.randint(1, T+1, (batch_size,), device=device)  # (batch_size,) 整数のt

    # 2. q_sampleで x_t, eps を生成
    x_t, eps = q_sample(x0, t)

    # 3. モデルで eps を予測
    eps_pred = model(x_t, t)

    # 4. 損失計算 (予測した eps と 本物の eps のMSE)
    loss = criterion(eps_pred, eps)

    # 5. バックプロパゲーション
    loss.backward()
    optimizer.step()

    losses.append(loss.item())

    if (epoch+1) % 500 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}")

# =====================================
# 学習曲線の可視化
# =====================================
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()

# =====================================
# サンプリング (逆拡散)
# =====================================
# 学習したモデルを使い、純ノイズ x_T からステップを減らしながら x_0 に近づける
model.eval()

# x_T を標準正規分布から生成
x_gen = torch.randn_like(x0)  # (10,1)と同じ形状のランダムノイズ

with torch.no_grad():
    for i in reversed(range(T)):
        t_curr = torch.tensor([i+1]*x0.size(0), device=device)  # t=(i+1)
        alpha_bar_t = alpha_bars[i]
        alpha_t = alphas[i]
        
        # モデルが推定した eps
        eps_pred = model(x_gen, t_curr)

        # 逆拡散ステップ (DDPM式)
        # μ_t = (x_t - (1 - α_t)/sqrt(1 - α_bar_t)*eps_pred) / sqrt(α_t)
        # ここでは簡易的に DDPM の1ステップ逆更新を近似
        if i > 0:
            alpha_bar_t_prev = alpha_bars[i-1]
        else:
            alpha_bar_t_prev = torch.tensor(1.0, device=device)  # t=1の前はα_bar=1とする
        
        mu = (x_gen - (1 - alpha_t)/torch.sqrt(1 - alpha_bar_t)*eps_pred) / torch.sqrt(alpha_t)
        
        if i > 0:
            # ストカスティックなステップ
            # 標準正規ノイズ
            z = torch.randn_like(x_gen)
            sigma = torch.sqrt((1 - alpha_t)*(1 - alpha_bar_t_prev)/(1 - alpha_bar_t))
            x_gen = mu + sigma * z
        else:
            # 最終ステップではノイズを加えない
            x_gen = mu

# x_gen がモデルが再構成した最終的なサンプル
x_gen = x_gen.cpu().numpy()

print("Generated Samples:", x_gen.squeeze())


RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x11 and 2x32)